# Week 6- Map Reduce and Apache Spark

**Objectives**: Today we are going to work with Apache Spark on AWS. Spark is one of the most popular "big data" platforms and supports both batch processing like Hadoop and streaming. We will also explore the basics of the MapReduce programming model that was the basis for Apache Hadoop. Today we will:
  
* Review MapReduce
* Review Spark and its place in the "big data" technology ecosystem
* Set up our Spark environment including PySpark
* Read data from S3 into an RDD
* Conduct some analyses using Spark

In [4]:
from ipyparallel import Client
c = Client()
c.ids

[0, 1, 2, 3, 4]

In [5]:
c.ids

[0, 1, 2, 3, 4]

In [6]:
dview=c[:]

In [12]:
%time map(lambda x:x**10, range(1000))

CPU times: user 1.51 ms, sys: 63 µs, total: 1.58 ms
Wall time: 1.53 ms


[0,
 1,
 1024,
 59049,
 1048576,
 9765625,
 60466176,
 282475249,
 1073741824,
 3486784401,
 10000000000,
 25937424601,
 61917364224,
 137858491849,
 289254654976,
 576650390625,
 1099511627776,
 2015993900449,
 3570467226624,
 6131066257801,
 10240000000000,
 16679880978201,
 26559922791424,
 41426511213649,
 63403380965376,
 95367431640625,
 141167095653376,
 205891132094649,
 296196766695424,
 420707233300201,
 590490000000000,
 819628286980801,
 1125899906842624,
 1531578985264449,
 2064377754059776,
 2758547353515625,
 3656158440062976,
 4808584372417849,
 6278211847988224,
 8140406085191601,
 10485760000000000,
 13422659310152401,
 17080198121677824,
 21611482313284249,
 27197360938418176,
 34050628916015625,
 42420747482776576,
 52599132235830049,
 64925062108545024,
 79792266297612001,
 97656250000000000,
 119042423827613001,
 144555105949057024,
 174887470365513049,
 210832519264920576,
 253295162119140625,
 303305489096114176,
 362033331456891249,
 430804206899405824,
 511116

In [11]:
%time dview.map_sync(lambda x: x**10, range(1000))

CPU times: user 11.7 ms, sys: 2.64 ms, total: 14.4 ms
Wall time: 12 ms


[0,
 1,
 1024,
 59049,
 1048576,
 9765625,
 60466176,
 282475249,
 1073741824,
 3486784401,
 10000000000,
 25937424601,
 61917364224,
 137858491849,
 289254654976,
 576650390625,
 1099511627776,
 2015993900449,
 3570467226624,
 6131066257801,
 10240000000000,
 16679880978201,
 26559922791424,
 41426511213649,
 63403380965376,
 95367431640625,
 141167095653376,
 205891132094649,
 296196766695424,
 420707233300201,
 590490000000000,
 819628286980801,
 1125899906842624,
 1531578985264449,
 2064377754059776,
 2758547353515625,
 3656158440062976,
 4808584372417849,
 6278211847988224,
 8140406085191601,
 10485760000000000,
 13422659310152401,
 17080198121677824,
 21611482313284249,
 27197360938418176,
 34050628916015625,
 42420747482776576,
 52599132235830049,
 64925062108545024,
 79792266297612001,
 97656250000000000,
 119042423827613001,
 144555105949057024,
 174887470365513049,
 210832519264920576,
 253295162119140625,
 303305489096114176,
 362033331456891249,
 430804206899405824,
 511116

In [22]:
%pwd

u'/Users/sellco/Documents/My_Documents/ASU/big_data/week_6'

In [28]:
from textblob import TextBlob
import codecs
with codecs.open('/Users/sellco/Documents/My_Documents/ASU/CIS 591/moby_dick.txt','r',encoding='utf8') as f:
    text = f.read()

In [32]:
words2 =TextBlob(text)

In [33]:
%time words2.np_counts

CPU times: user 15min 29s, sys: 11.2 s, total: 15min 40s
Wall time: 15min 31s


defaultdict(int,
            {u'cussed': 1,
             u'new-mown hay': 1,
             u'fresh wind': 1,
             u'yellow': 1,
             u'real blanket': 1,
             u'sinful men': 2,
             u'general reference': 1,
             u'small bits': 1,
             u"'s dinner": 1,
             u'full terms': 2,
             u'rueful sight': 1,
             u'pagan kinsmen': 1,
             u'famed greek fire': 1,
             u'deacon deuteronomy': 3,
             u'granting': 2,
             u"rope 's durability": 1,
             u'eternal democracy': 1,
             u'heavy cannon': 1,
             u'russian campaign': 1,
             u'new beaver hat': 1,
             u'black faces': 1,
             u'mysterious fatality': 1,
             u'l100,000': 2,
             u'having his spoutings out': 1,
             u'applicable taxes': 1,
             u'good look': 2,
             u'sicilian sailor': 2,
             u'silent steersman': 1,
             u'steel goblets': 